In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score

In [71]:
def generate_data(no_points):
    X = np.zeros(shape=(no_points, 2))
    Y = np.zeros(shape=no_points)
    for ii in range(no_points):
        X[ii, 0] = random.randint(0,100)
        X[ii, 1] = random.randint(0,100)
        if X[ii, 0]+X[ii, 1] > 100:
            Y[ii] = 1 
        else:
            Y[ii] = -1
    return X, Y


X, y = generate_data(1000)
X = np.hstack((X, (np.ones((X.shape[0], 1)))))  # Assuming this step is still needed

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)
y_tensor = (y_tensor + 1) // 2  # Adjust labels if needed

# Create TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
num_epochs = 1000

In [72]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Adjust input_size here
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [73]:
# Assuming your dataset has 3 features per input
# Check if a GPU is available and select it, otherwise fall back to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

model = MLP(input_size=3, hidden_size=500, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam optimizer with learning rate 0.001


Using device: cuda


In [74]:
num_epochs = 1000  # Example, set the number of epochs as needed
total_step = len(train_loader)  # The number of batches in the train_loader

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Verbose output
        if (i + 1) % 100 == 0:  # Adjust the print frequency as needed
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')



Epoch [1/1000], Step [10/13], Loss: 7.9434
Epoch [2/1000], Step [10/13], Loss: 3.2635
Epoch [3/1000], Step [10/13], Loss: 1.8610
Epoch [4/1000], Step [10/13], Loss: 0.9373
Epoch [5/1000], Step [10/13], Loss: 0.7259
Epoch [6/1000], Step [10/13], Loss: 0.5245
Epoch [7/1000], Step [10/13], Loss: 0.6127
Epoch [8/1000], Step [10/13], Loss: 0.5850
Epoch [9/1000], Step [10/13], Loss: 0.6702
Epoch [10/1000], Step [10/13], Loss: 0.5705
Epoch [11/1000], Step [10/13], Loss: 0.5648
Epoch [12/1000], Step [10/13], Loss: 0.5635
Epoch [13/1000], Step [10/13], Loss: 0.5485
Epoch [14/1000], Step [10/13], Loss: 0.4963
Epoch [15/1000], Step [10/13], Loss: 0.5989
Epoch [16/1000], Step [10/13], Loss: 0.5411
Epoch [17/1000], Step [10/13], Loss: 0.5564
Epoch [18/1000], Step [10/13], Loss: 0.5427
Epoch [19/1000], Step [10/13], Loss: 0.5433
Epoch [20/1000], Step [10/13], Loss: 0.4602
Epoch [21/1000], Step [10/13], Loss: 0.5134
Epoch [22/1000], Step [10/13], Loss: 0.4715
Epoch [23/1000], Step [10/13], Loss: 0.61

In [75]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network: {100 * correct / total} %')


Accuracy of the network: 98.5 %
